<a href="https://colab.research.google.com/github/fedoua-ops/-Extraction-de-donn-es-via-OCR/blob/main/OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytesseract
!pip install Pillow
!pip install opencv-python
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 25.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyl

In [11]:
!sudo apt-get update
!sudo apt-get install -y tesseract-ocr
!sudo apt-get install -y tesseract-ocr-fra


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 47 not upgraded.
Reading pac

In [5]:
!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,125 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,376 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-securit

In [28]:

import pytesseract
from pytesseract import Output
import easyocr
import cv2
import json

# Initialisation EasyOCR reader
reader = easyocr.Reader(['fr'])

def extract_text_tesseract(image):
    data = pytesseract.image_to_data(image, output_type=Output.DICT)
    extracted_data = []
    for i in range(len(data['text'])):
        if int(data['conf'][i]) > 0:
            x, y, width, height = data['left'][i], data['top'][i], data['width'][i], data['height'][i]
            text = data['text'][i]
            confidence = int(data['conf'][i])
            extracted_data.append({
                'text': text,
                'confidence': confidence,
                'position': {'x': x, 'y': y, 'width': width, 'height': height}
            })
    return extracted_data

def extract_text_easyocr(image_path):
    results = reader.readtext(image_path)
    extracted_data = []
    for (bbox, text, confidence) in results:
        (top_left, top_right, bottom_right, bottom_left) = bbox
        x = int(top_left[0])
        y = int(top_left[1])
        width = int(top_right[0] - top_left[0])
        height = int(bottom_left[1] - top_left[1])
        extracted_data.append({
            'text': text,
            'confidence': confidence,
            'position': {'x': x, 'y': y, 'width': width, 'height': height}
        })
    return extracted_data

def combine_results(tesseract_data, easyocr_data):
    combined_data = tesseract_data + easyocr_data
    combined_data = sorted(combined_data, key=lambda x: (-x['confidence'], x['position']['y'], x['position']['x']))
    return combined_data

image_path = 'Genova.png'
image = cv2.imread(image_path)

# Extract text using Tesseract and EasyOCR
tesseract_data = extract_text_tesseract(image)
easyocr_data = extract_text_easyocr(image_path)

# Combine the results
combined_data = combine_results(tesseract_data, easyocr_data)


print("Toutes les données extraites :", json.dumps(combined_data, indent=4, ensure_ascii=False))

# Filter relevant cities
cities = ['Gênes', 'Camaldoli']
filtered_data = [entry for entry in combined_data if any(city.lower() in entry['text'].lower() for city in cities)]


print("Données filtrées :", json.dumps(filtered_data, indent=4, ensure_ascii=False))

# Save data to a JSON file
output_path = 'output.json'
with open(output_path, 'w', encoding='utf-8') as json_file:
    json.dump(combined_data, json_file, ensure_ascii=False, indent=4)

Toutes les données extraites : [
    {
        "text": " ",
        "confidence": 95,
        "position": {
            "x": 0,
            "y": 0,
            "width": 739,
            "height": 468
        }
    },
    {
        "text": "",
        "confidence": 95,
        "position": {
            "x": 128,
            "y": 531,
            "width": 142,
            "height": 15
        }
    },
    {
        "text": "Gênes",
        "confidence": 0.9999588430244524,
        "position": {
            "x": 284,
            "y": 218,
            "width": 76,
            "height": 30
        }
    },
    {
        "text": "Camaldoli",
        "confidence": 0.9999399176155841,
        "position": {
            "x": 593,
            "y": 157,
            "width": 60,
            "height": 16
        }
    },
    {
        "text": "Bacino",
        "confidence": 0.9338988177371237,
        "position": {
            "x": 234,
            "y": 320,
            "width": 35,
            "hei

In [24]:
import cv2
import pytesseract
import pandas as pd
import os


image_path = 'Extrait_IQOA_data.png'

os.environ['TESSDATA_PREFIX'] = '/usr/share/tesseract-ocr/4.00/tessdata/'

image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)


if image is None:
    raise FileNotFoundError(f"Image not found at path: {image_path}")

# Prétraitement de l'iamge
blur = cv2.GaussianBlur(image, (5, 5), 0)
_, binary = cv2.threshold(blur, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# les contours
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

contours = [c for c in contours if cv2.contourArea(c) > 1000]

contours = sorted(contours, key=lambda c: (cv2.boundingRect(c)[1], cv2.boundingRect(c)[0]))

# Initialisation
table_data = []

# Extraction
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    roi = image[y:y+h, x:x+w]
    text = pytesseract.image_to_string(roi, config='--psm 6', lang='fra').strip()
    table_data.append({'text': text, 'position': {'x': x, 'y': y, 'width': w, 'height': h}})

# Organisation
rows = {}
for data in table_data:
    x, y, width, height = data['position'].values()
    row_key = y //
    col_key = x //

    if row_key not in rows:
        rows[row_key] = {}

    rows[row_key][col_key] = data['text']

#DataFrame
structured_data = []
for row_key in sorted(rows.keys()):
    row = []
    for col_key in sorted(rows[row_key].keys()):
        row.append(rows[row_key][col_key])
    structured_data.append(row)

df = pd.DataFrame(structured_data)

df

# Sauvegarde
output_csv_path = 'extracted_table.csv'
df.to_csv(output_csv_path, index=False, header=False)

print(f"Tableau extrait et sauvegardé sous forme de CSV à : {output_csv_path}")






Tableau extrait et sauvegardé sous forme de CSV à : extracted_table.csv
